# Get Historical Data

In [1]:
import pfeed as pe

pe.__version__

'0.0.1.dev12'

## Create Data Feeds

 ```{seealso}
Pick your favorite data tool, e.g. `pandas`, `polars` etc, see [Supported Data Tools](../../supported-data-tools.md) for more information.
```

In [2]:
bybit_feed = pe.BybitFeed(data_tool='polars')
yfinance_feed = pe.YahooFinanceFeed(data_tool='pandas')

## Get Historical Data from Bybit

In [3]:
bybit_df = bybit_feed.get_historical_data(
    'BTC_USDT_PERP',
    rollback_period='2d',  # rollback 2 days
    resolution='1m',  # 1-minute data  
)

2024-09-01T21:46:40+0800.219 | DEBUG | bybit_data | Removed temporary directory: /Users/stephenyau/pfund.ai/pfeed/.local/data/bybit_temp_7xg66x25 | fn:_prepare_temp_dir ln:172
2024-09-01T21:46:40+0800.429 | DEBUG | bybit_data | failed to extract BYBIT minute data from local path /Users/stephenyau/pfund.ai/pfeed/.local/data/bybit/historical/minute/BTC_USDT_PERP/BTC_USDT_PERP_2024-08-30.parquet | fn:extract_data ln:128
2024-09-01T21:46:40+0800.477 | DEBUG | bybit_data | failed to extract BYBIT minute data from MinIO object bybit/historical/minute/BTC_USDT_PERP/BTC_USDT_PERP_2024-08-30.parquet | fn:extract_data ln:136
2024-09-01T21:46:40+0800.517 | DEBUG | bybit_data | extracted BYBIT raw_tick data from local path /Users/stephenyau/pfund.ai/pfeed/.local/data/bybit/historical/raw_tick/BTC_USDT_PERP/BTC_USDT_PERP_2024-08-30.parquet | fn:extract_data ln:125
2024-09-01T21:46:40+0800.519 | INFO | bybit_data | No local BYBIT data found with dtype='minute', switched to find "raw_tick" data inste

By calling just **one line of code** above, now you can play with the clean data returned.

In [4]:
bybit_df.collect().tail(3)

ts,product,resolution,open,high,low,close,volume
datetime[ns],str,str,f64,f64,f64,f64,f64
2024-08-31 23:57:00,"""BTC_USDT_PERP""","""1m""",58950.4,58950.4,58946.9,58947.0,1.155
2024-08-31 23:58:00,"""BTC_USDT_PERP""","""1m""",58947.0,58947.0,58929.5,58946.7,29.341
2024-08-31 23:59:00,"""BTC_USDT_PERP""","""1m""",58946.6,58951.3,58944.4,58944.4,4.137


## Get Historical Data from Yahoo Finance

```{note}
yfinance_feed actually uses the [yfinance](https://github.com/ranaroussi/yfinance) library to fetch data, so you can still use kwargs that are supported by yfinance.
```

In [6]:
yfinance_kwargs = {}  # please refer to yfinance's documentation for the supported kwargs
df = yfinance_feed.get_historical_data(
    'TSLA',
    resolution='1d',  # 1-day data
    start_date='2024-01-01',
    end_date='2024-01-31',
    **yfinance_kwargs
)
df.head(3)

,ts,symbol,resolution,open,high,low,close,volume,dividends,splits
0,2024-01-02 05:00:00,TSLA,1d,250.080002,251.250000,244.410004,248.419998,104654200,0.0,0.0
1,2024-01-03 05:00:00,TSLA,1d,244.979996,245.679993,236.320007,238.449997,121082600,0.0,0.0
2,2024-01-04 05:00:00,TSLA,1d,239.250000,242.699997,237.729996,237.929993,102629300,0.0,0.0


```{seealso}
See [Supported Data Sources](../../supported-data-sources.md) to find out what data sources are supported.
```